In [87]:
import os
import sys
import json
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [88]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "baryshev konstantin") 

spark = SparkSession.builder.config(conf=conf).appName("baryshev konstantin").getOrCreate()

In [89]:
#!hdfs dfs -ls /labs/laba01/ml-100k

In [59]:
#!hdfs dfs -cat /labs/laba01/ml-100k/u.item | head

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0
6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0
8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Babe%20(1995)|0|0|0|0|1

In [90]:
from pyspark.sql.types import StructType, StructField, IntegerType

In [91]:
schema = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("rating", IntegerType()),
    StructField("timestamp", IntegerType())
])

In [92]:
df_marks = spark.read\
          .schema(schema)\
          .format("csv")\
          .option("sep", "\t")\
          .load("/lectures/lecture02/data/ml-100k/u.data")

In [93]:
df_marks.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [104]:
pers_film = df_marks.where("item_id = 318")\
                    .groupby('rating')\
                    .count()\
                    .withColumnRenamed("count", "_count")\
                    .orderBy('rating')

In [105]:
pers_film.show() 

+------+------+
|rating|_count|
+------+------+
|     1|     4|
|     2|     6|
|     3|    23|
|     4|    79|
|     5|   186|
+------+------+



In [106]:
all_film = df_marks.groupby('rating')\
                    .count()\
                    .withColumnRenamed("count", "_count")\
                    .orderBy('rating')

In [107]:
all_film.show()

+------+------+
|rating|_count|
+------+------+
|     1|  6110|
|     2| 11370|
|     3| 27145|
|     4| 34174|
|     5| 21201|
+------+------+



In [116]:
with open('/data/home/konstantin.baryshev/lab01.json', 'w') as f:
    json.dump({'hist_film': [row._count for row in pers_film.collect()], "hist_all": [row._count for row in all_film.collect()]}, f)

In [118]:
spark.stop()